In [332]:
import numpy as np 
import pandas as pd
import re
import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [333]:
credits=pd.read_csv(r'F:\NLP project\tmdb_5000_credits.csv\tmdb_5000_credits.csv',encoding_errors = 'ignore')
movies=pd.read_csv(r'F:\NLP project\tmdb_5000_movies.csv\tmdb_5000_movies.csv',encoding_errors = 'ignore')


In [334]:
credits_column_renamed=credits.rename(columns={"movie_id":"id"})
movies_df_merged=movies.merge(credits_column_renamed,on='id')
movies_clean=movies_df_merged[['id','original_title','overview','label']]


In [335]:
col=movies_clean.columns
res=[]
for c in col:
    d=movies_clean[col].values.tolist()
    res.append(d)


In [336]:
stopwords = []
corpus=[]
movie_tfidfvector=[]
with open("stopword.txt", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        stopwords.append(line.strip("\n").strip())
countlist = []
for i in range(0,1470):
    with open(r"F:\NLP project\movie/"+str(i)+".txt", "r",encoding='utf-8') as f:    
        data = f.read()   
        
    text = re.sub(r"'s", " 's", data)
    text = re.sub(r"'ve", " 've", data)
    text = re.sub(r"n't", " n't", data)
    text = re.sub(r"'re", " 're", data)
    text = re.sub(r"'d", " 'd", data)
    text = re.sub(r"'ll", " 'll", data)
    text = re.sub(r"[^A-Za-z]", " ", data)
    do_lower_case=True
    if do_lower_case:
        text=text.strip().lower()
    tokens = text.split()
    text_words = [token for token in tokens if token not in stopwords]
    corpus.append(text_words)

In [337]:
def read_corpus(corpus):
    for i, line in enumerate(corpus):
        yield gensim.models.doc2vec.TaggedDocument(line, [i])
train_corpus = list(read_corpus(corpus))

In [338]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40,dm=1)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
movie_vector=[]
for i in range(0,1470):
    movie_vector.append(model.infer_vector(corpus[i]))
movie_vector

[array([-0.19252907,  0.189787  ,  0.05148498, -0.12607795,  0.23412934,
         0.2299487 , -0.13475607, -0.08887023, -0.4050601 , -0.09241518,
         0.02209557, -0.0428364 , -0.02235529,  0.07477336, -0.03934803,
         0.01166928,  0.11724058,  0.04651084, -0.19734938, -0.34693903,
         0.14348179, -0.04716173,  0.20067969, -0.36558947,  0.23009701,
         0.12599877, -0.13864776, -0.17812775, -0.11636318, -0.02472833,
         0.1264014 ,  0.11476234, -0.05955012,  0.27690873, -0.12997766,
         0.09177572, -0.01266274,  0.10874573, -0.09405275, -0.3627671 ,
         0.13624683, -0.1393134 ,  0.13567705,  0.05429282,  0.5048592 ,
        -0.17139396, -0.11053891, -0.11319625, -0.15065072,  0.02982347],
       dtype=float32),
 array([-0.1361797 ,  0.1589996 ,  0.08530085, -0.15068115,  0.0738517 ,
        -0.06187176, -0.09294415,  0.11511362, -0.28154585, -0.0657443 ,
        -0.08497594,  0.063461  , -0.05417127,  0.0377701 , -0.08520346,
        -0.07752385,  0.109

In [339]:
indices=pd.Series(movies_clean.index,index=movies_clean.original_title)
def euclideandistance(x,y):
    return np.sqrt(np.sum(np.square(x-y)))

In [340]:
vector=model.infer_vector(corpus[1])
sims = model.docvecs.most_similar([vector],topn=6)
sims

C:\Users\ASUS\AppData\Local\Temp\ipykernel_23156\3011338544.py:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = model.docvecs.most_similar([vector],topn=6)


[(778, 0.8831777572631836),
 (581, 0.871728777885437),
 (1044, 0.8374736905097961),
 (1341, 0.8341088891029358),
 (1034, 0.8263283371925354),
 (454, 0.8251809477806091)]

In [341]:
def recommend(a):
    vector=model.infer_vector(corpus[a])
    sims = model.docvecs.most_similar([vector],topn=10)
    movies_indices=[]
    for i in sims:
        movies_indices.append(i[0])
    series=movies_clean.label.iloc[movies_indices]
    return series.values

In [342]:
acclis=[]
for i in range(0,100):
    a=0
    for j in recommend(i):
        if j==movies_clean['label'][i]:
            a=a+1
    accuracy=a/10
    acclis.append(accuracy)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_23156\2514842605.py:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = model.docvecs.most_similar([vector],topn=10)


In [343]:
acc=sum(acclis)/len(acclis)
print('%.3f' % acc)

0.445
